# Starbucks Capstone Challenge

# Part 1 - DEFINE

### ---- 1 Define the problem ----

# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [2]:
#import your libraries
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

#your info here
__author__ = "Sha Brown"
__email__ = "sha821@gmail.com"

## Part 2 - DISCOVER

### ---- 2 Load the data ----

In [3]:
# load the data into a Pandas dataframe
# read in the json files
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True)

In [4]:
# display the first 10 rows of the dataframe
portfolio.head(10)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [5]:
id_length = lambda x: len(x)
length = portfolio['id'].apply(id_length)
length

0    32
1    32
2    32
3    32
4    32
5    32
6    32
7    32
8    32
9    32
Name: id, dtype: int64

In [6]:
# print dataframe shape
portfolio.shape

(10, 6)

In [7]:
# display the first 5 rows of the dataframe
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [8]:
# print dataframe shape
profile.shape

(17000, 5)

In [9]:
# display the first 5 rows of the dataframe
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [10]:
# print dataframe shape
transcript.shape

(306534, 4)

In [11]:
transcript['value'] = (transcript['value']
                       .astype('str')
                       .str.replace('{','')
                       .str.replace('}','')
                      )

In [12]:
transcript['value'] = transcript['value'].str.split(':')

In [13]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,"['offer id', '9b98b8c7a33c4b65b9aebfe6a799e6d9']",0
1,a03223e636434f42ac4c3df47e8bac43,offer received,"['offer id', '0b1e1539f2cc45b7b9fa7c272da2e1d7']",0
2,e2127556f4f64592b11af22de27a7932,offer received,"['offer id', '2906b810c7d4411798c6938adc9daaa5']",0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,"['offer id', 'fafdcd668e3743c1bb461111dcafc2a4']",0
4,68617ca6246f4fbc85e91a2a49552598,offer received,"['offer id', '4d5c57ea9a6940dd891ad53e9dbe8da0']",0


In [14]:
transcript.loc[0, 'value'][0]

"'offer id'"

In [15]:
transcript.loc[0, 'value'][1]

" '9b98b8c7a33c4b65b9aebfe6a799e6d9'"

In [16]:
extract_offer = lambda x: x[0]
extract_value = lambda x: x[1]

transcript['offer_or_amount'] = transcript['value'].apply(extract_offer)
transcript['id'] = transcript['value'].apply(extract_value)

In [17]:
transcript = transcript.drop('value', axis=1)

In [18]:
transcript.head()

,person,event,time,offer_or_amount,id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,'offer id','9b98b8c7a33c4b65b9aebfe6a799e6d9'
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,'offer id','0b1e1539f2cc45b7b9fa7c272da2e1d7'
2,e2127556f4f64592b11af22de27a7932,offer received,0,'offer id','2906b810c7d4411798c6938adc9daaa5'
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,'offer id','fafdcd668e3743c1bb461111dcafc2a4'
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,'offer id','4d5c57ea9a6940dd891ad53e9dbe8da0'


In [19]:
transcript['offer_or_amount'] = (transcript['offer_or_amount']
                       .str.replace('_', '')
                       .str.replace("'", "")
                       )

In [20]:
transcript['id'] = transcript['id'].str.replace("'", "")
                             

In [21]:
# change column name from 'person' to 'customer_id'
transcript.rename(columns={'person': 'customer_id', 'offer':'offer_id'}, inplace=True)

#### Merge Data

In [22]:
# change column name from 'id' to 'customer_id'
profile.rename(columns={'id': 'customer_id'}, inplace=True)

In [23]:
# first we will merge transcript with profile 
merged_df = transcript.merge(profile, on='customer_id', how='left')

In [24]:
merged_df.head()

,customer_id,event,time,offer_or_amount,id,gender,age,became_member_on,income
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,20170509,100000.0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,offer id,0b1e1539f2cc45b7b9fa7c272da2e1d7,None,118,20170804,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,offer id,2906b810c7d4411798c6938adc9daaa5,M,68,20180426,70000.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,offer id,fafdcd668e3743c1bb461111dcafc2a4,None,118,20170925,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,offer id,4d5c57ea9a6940dd891ad53e9dbe8da0,None,118,20171002,NaN


In [25]:
# check the first few rows of the dataframe
merged_df['id'].value_counts().head(20)

 2298d6c36e964ae4a3e7e9706d1fb8c2            14983
 fafdcd668e3743c1bb461111dcafc2a4            14924
 4d5c57ea9a6940dd891ad53e9dbe8da0            14891
 f19421c1d4aa40978ebb69ca19b0e20d            14835
 ae264e3637204a6fb9bb56bc8210ddfd            14374
 5a8bc65990b245e5a138643cd4eb9837            14305
 9b98b8c7a33c4b65b9aebfe6a799e6d9            11848
 3f207df678b143eea3cee63160fa8bed            11761
 2906b810c7d4411798c6938adc9daaa5            11750
 0b1e1539f2cc45b7b9fa7c272da2e1d7            10331
 fafdcd668e3743c1bb461111dcafc2a4, reward     5317
 2298d6c36e964ae4a3e7e9706d1fb8c2, reward     5156
 9b98b8c7a33c4b65b9aebfe6a799e6d9, reward     4354
 f19421c1d4aa40978ebb69ca19b0e20d, reward     4296
 2906b810c7d4411798c6938adc9daaa5, reward     4017
 ae264e3637204a6fb9bb56bc8210ddfd, reward     3688
 0b1e1539f2cc45b7b9fa7c272da2e1d7, reward     3420
 4d5c57ea9a6940dd891ad53e9dbe8da0, reward     3331
 0.05                                          431
 0.66                          

In [26]:
# remove"," and "reward" from offer_value column
merged_df['id'] = merged_df['id'].str.replace(",", "").str.replace('reward', '').str.replace(" ",'')

In [27]:
merged_df['id'].value_counts().head(10)

fafdcd668e3743c1bb461111dcafc2a4    20241
2298d6c36e964ae4a3e7e9706d1fb8c2    20139
f19421c1d4aa40978ebb69ca19b0e20d    19131
4d5c57ea9a6940dd891ad53e9dbe8da0    18222
ae264e3637204a6fb9bb56bc8210ddfd    18062
9b98b8c7a33c4b65b9aebfe6a799e6d9    16202
2906b810c7d4411798c6938adc9daaa5    15767
5a8bc65990b245e5a138643cd4eb9837    14305
0b1e1539f2cc45b7b9fa7c272da2e1d7    13751
3f207df678b143eea3cee63160fa8bed    11761
Name: id, dtype: int64

In [28]:
# print out the shape of the dataframe
merged_df.shape

(306534, 9)

In [29]:
merged_df['offer_or_amount'].value_counts()

amount      138953
offer id    134002
offerid      33579
Name: offer_or_amount, dtype: int64

In [30]:
merged_df['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [31]:
merged_df = merged_df.merge(portfolio, on='id', how='left')

In [32]:
merged_df.head()

,customer_id,event,time,offer_or_amount,id,gender,age,became_member_on,income,reward,channels,difficulty,duration,offer_type
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,20170509,100000.0,5.0,"[web, email, mobile]",5.0,7.0,bogo
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,offer id,0b1e1539f2cc45b7b9fa7c272da2e1d7,None,118,20170804,NaN,5.0,"[web, email]",20.0,10.0,discount
2,e2127556f4f64592b11af22de27a7932,offer received,0,offer id,2906b810c7d4411798c6938adc9daaa5,M,68,20180426,70000.0,2.0,"[web, email, mobile]",10.0,7.0,discount
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,offer id,fafdcd668e3743c1bb461111dcafc2a4,None,118,20170925,NaN,2.0,"[web, email, mobile, social]",10.0,10.0,discount
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,offer id,4d5c57ea9a6940dd891ad53e9dbe8da0,None,118,20171002,NaN,10.0,"[web, email, mobile, social]",10.0,5.0,bogo


### ---- 3 Clean the data ----

In [33]:
# extract all available channels
channels_set = set()
for channel in portfolio.channels:
    channels_set.update(set(channel))
channels_set

{'email', 'mobile', 'social', 'web'}

In [35]:
# create new columns corresponding to channels
for i in channels_set:
    merged_df[i] = merged_df['channels'].astype('str').apply(lambda x: i in x).map({True:1, False: 0})
merged_df.head()

,customer_id,event,time,offer_or_amount,id,gender,age,became_member_on,income,reward,channels,difficulty,duration,offer_type,social,web,mobile,email
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,20170509,100000.0,5.0,"[web, email, mobile]",5.0,7.0,bogo,0,1,1,1
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,offer id,0b1e1539f2cc45b7b9fa7c272da2e1d7,None,118,20170804,NaN,5.0,"[web, email]",20.0,10.0,discount,0,1,0,1
2,e2127556f4f64592b11af22de27a7932,offer received,0,offer id,2906b810c7d4411798c6938adc9daaa5,M,68,20180426,70000.0,2.0,"[web, email, mobile]",10.0,7.0,discount,0,1,1,1
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,offer id,fafdcd668e3743c1bb461111dcafc2a4,None,118,20170925,NaN,2.0,"[web, email, mobile, social]",10.0,10.0,discount,1,1,1,1
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,offer id,4d5c57ea9a6940dd891ad53e9dbe8da0,None,118,20171002,NaN,10.0,"[web, email, mobile, social]",10.0,5.0,bogo,1,1,1,1


### ---- 4 Explore the data (EDA) ----

In [ ]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

### ---- 5 Establish a baseline ----

In [ ]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

### ---- 6 Hypothesize solution ----

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### ---- 8 Create models ----

In [ ]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [ ]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [ ]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data